In [1]:
from faker import Faker
import random
import time
import csv
import pandas as pd
import string 
import json 
faker = Faker()
# Load data from CSV
def load_csv(filename):
    return pd.read_csv(filename)
# Generate Unix timestamp
def generate_timestamp():
    return int(time.time())
# Write data to CSV
def write_to_csv(filename, fieldnames, data):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerows(data)
# Function to generate nullable values
def nullable_value(generator, null_probability=0.2):
    if random.random() < null_probability:
        return None
    return generator()
# Unique value generator
def unique_value(existing_set, generator):
    value = generator()
    while value in existing_set:
        value = generator()
    existing_set.add(value)
    return value



In [4]:
itinerary_data = load_csv('level_2/itinerary.csv')

itinerary_lookup = list(itinerary_data[['ItineraryID']].itertuples(index = False, name = None))

In [13]:
from datetime import datetime, timedelta
booking_id_counter = 0
unique_booking_refs = set()
currency_codes = [
    "AED", "AFN", "ALL", "AMD", "AOA", "ARS", "AUD", "AWG", "AZN", "BAM",
    "BBD", "BDT", "BGN", "BHD", "BIF", "BMD", "BND", "BOB", "BOV", "BRL",
    "BSD", "BTN", "BWP", "BYN", "BZD", "CAD", "CDF", "CHE", "CHF", "CHW",
    "CLF", "CLP", "CNY", "COP", "COU", "CRC", "CUC", "CUP", "CVE", "CZK",
    "DJF", "DKK", "DOP", "DZD", "EGP", "ERN", "ETB", "EUR", "FJD", "FKP",
    "GBP", "GEL", "GHS", "GIP", "GMD", "GNF", "GTQ", "GYD", "HKD", "HNL",
    "HRK", "HTG", "HUF", "IDR", "ILS", "INR", "IQD", "IRR", "ISK", "JMD",
    "JOD", "JPY", "KES", "KGS", "KHR", "KMF", "KPW", "KRW", "KWD", "KYD",
    "KZT", "LAK", "LBP", "LKR", "LRD", "LSL", "LYD", "MAD", "MDL", "MGA",
    "MKD", "MMK", "MNT", "MOP", "MRU", "MUR", "MVR", "MWK", "MXN", "MXV",
    "MYR", "MZN", "NAD", "NGN", "NIO", "NOK", "NPR", "NZD", "OMR", "PAB",
    "PEN", "PGK", "PHP", "PKR", "PLN", "PYG", "QAR", "RON", "RSD", "RUB",
    "RWF", "SAR", "SBD", "SCR", "SDG", "SEK", "SGD", "SHP", "SLE", "SLL",
    "SOS", "SRD", "SSP", "STN", "SVC", "SYP", "SZL", "THB", "TJS", "TMT",
    "TND", "TOP", "TRY", "TTD", "TWD", "TZS", "UAH", "UGX", "USD", "USN",
    "UYI", "UYU", "UZS", "VED", "VES", "VND", "VUV", "WST", "XAF", "XAG",
    "XAU", "XBA", "XBB", "XBC", "XBD", "XCD", "XDR", "XOF", "XPD", "XPF"
]

def generate_unique_booking_ref():
    while True:
        ref = faker.bothify(text='????######').upper()
        if ref not in unique_booking_refs:
            unique_booking_refs.add(ref)
            return ref

def generate_booking_status():
    return random.choices(
        ['Confirmed', 'Cancelled'],
        weights=[0.7, 0.3],
        k=1
    )[0]

def generate_booking(itineray_lookup, num_records):
    global booking_id_counter
    bookings = []

    for _ in range(num_records):
        booking_id_counter += 1
        booking_date = faker.date_between(start_date='-10y', end_date='today')
        timestamp = int(datetime.now().timestamp())

        bookings.append({
            'BookingID': booking_id_counter,
            'BookingReference': generate_unique_booking_ref(),
            'ItineraryID': random.choice(itinerary_lookup)[0],
            'BookingDate': booking_date.strftime('%Y-%m-%d'),
            'BookingStatus': generate_booking_status(),
            'TotalAmount': round(random.uniform(50, 3000), 2),
            'Currency': random.choice(currency_codes),
            'SalesChannel': random.choice(['Website', 'MobileApp', 'Agency']),
            'BookingAgentID': random.choice([None, random.randint(1000, 9000)]),
            'CreatedAt': timestamp,
            'UpdatedAt': timestamp
        })

    return bookings
def write_to_csv(filename, fieldnames, data):
    import csv
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Ví dụ sử dụng
num_records = int(input("Nhập số lượng cần sinh: "))
booking_data = generate_booking(itinerary_lookup,num_records)  # sinh 200 booking

write_to_csv('level_3/booking.csv', booking_data[0].keys(), booking_data)
print("✅ Đã ghi xong file booking.csv")


✅ Đã ghi xong file booking.csv


In [21]:
df_capacity = load_csv('level_2/aircraft_lookup.csv')
df_capacity
df_aircraft = load_csv('level_2/aircraft.csv')
df_merged = df_aircraft.merge(df_capacity[['ICAO', 'Capacity']], left_on='ICAOTypeCode', right_on='ICAO', how='left')



In [24]:
df_merged
df_merged.to_csv('level_2/aircraft_with_Capacity.csv')

In [41]:
df = pd.read_csv('level_2/aircraft_with_Capacity.csv')
aircraftID_data = df.to_dict(orient='records')

In [49]:
current_seat_id = 0
def generate_seat_id():
    global current_seat_id 
    current_seat_id  +=1
    return current_seat_id
def generate_aircraft_seat_by_capacity(aircraft_data):
    seats = []
    seat_types = {
                    "A": "Window", "F": "Window",
                    "C": "Aisle", "D": "Aisle",
                    "B": "Middle", "E": "Middle"
                }
    cabin_classes = ['Economy', 'Premium economy', 'Business class', 'First class']

    for aircraft in aircraft_data:
        aircraft_id = aircraft['AircraftID']
        capacity = aircraft['Capacity']  # số ghế cần tạo cho máy bay này

        for seat_num in range(1, capacity + 1):
            letter = chr(65 + (seat_num - 1) % 6 )
            seat_number = f"{(seat_num - 1) // 6 + 1}{letter}"  # Ví dụ: ghế 1A, 1B,...
            seat_type = {
                    "A": "Window", "F": "Window",
                    "C": "Aisle", "D": "Aisle",
                    "B": "Middle", "E": "Middle"
                }[letter]  # Hoặc tự logic theo vị trí ghế
            cabin_class = random.choices(cabin_classes, weights = [0.8, 0.5, 0.12, 0.3], k = 1)[0]  # Có thể dựa trên vị trí hoặc tỷ lệ

            seat = {
                'SeatID': generate_seat_id(),
                'AircraftID': aircraft_id,
                'SeatNumber': seat_number,
                'SeatType': seat_type,
                'CabinClass': cabin_class,
                'Characteristics': json.dumps(random.sample(characteristics, random.randint(1, 3))),
                'IsBlocked': random.choices([True, False], weights=[10, 90])[0],
                'IsExitRow': random.choices([True, False], weights=[15, 85])[0],
                'CreatedAt': generate_timestamp(),
                'UpdatedAt': generate_timestamp()
            }
            seats.append(seat)
    return seats
aircraft_seats = generate_aircraft_seat_by_capacity(aircraftID_data)
write_to_csv('level_3/aircraft_seat.csv', aircraft_seats[0].keys(), aircraft_seats)
print("Đã ghi file thành công")


Đã ghi file thành công


In [ ]:

# Load aircraft data from the uploaded CSV file
aircraftID_data = load_csv('level_2/aircraft.csv')

# Convert the data into a list of tuples (AirlinesID)
aircraftID_lookup = list(aircraftID_data[['AircraftID']].itertuples(index=False, name=None))

In [ ]:
import random
from datetime import datetime, timedelta

def generate_operational_flight_legs(flight_data, num_legs, airports_lookup, airlines_lookup, aircraft_lookup):
    legs = []
    leg_id_counter = 1
    now = datetime.utcnow()
    statuses = ['Scheduled', 'Delayed', 'Cancelled', 'Departed', 'Landed']

    for _ in range(num_legs):
        use_null_flight = random.random() < 0.3  # 30% là NULL

        if not use_null_flight and flight_data:
            flight = random.choice(flight_data)

            flight_id = flight['FlightID']
            dep_airport = flight['DepartureAirportID']
            arr_airport = flight['ArrivalAirportID']
            flight_number = flight['FlightNumber']
        else:
            flight = random.choice(flight_data)
            flight_id = None
            dep_airport = flight['DepartureAirportID']
            arr_airport = random.choice([a[0] for a in airports_lookup if a[0] != dep_airport])
            flight_number = flight['FlightNumber'] + 'X'
            if arr_airport != flight['ArrivalAirportID']:
                flight_number += 'CD'

        leg_date = now.date() + timedelta(days=random.randint(-3, 7))
        dep_time = now + timedelta(hours=random.randint(1, 12))
        arr_time = dep_time + timedelta(minutes=random.randint(60, 240))
        timestamp = int(now.timestamp())

        legs.append({
            'LegID': leg_id_counter,
            'FlightID': flight_id,
            'LegDate': leg_date.strftime("%Y-%m-%d"),
            'ScheduledDepartureDateTime': dep_time.strftime("%Y-%m-%d %H:%M:%S"),
            'ScheduledArrivalDateTime': arr_time.strftime("%Y-%m-%d %H:%M:%S"),
            'DepartureAirportID': dep_airport,
            'ArrivalAirportID': arr_airport,
            'OperatingAirlineID': random.choice(airlines_lookup)[0],
            'MarketingAirlineID': random.choice([None, random.choice(airlines_lookup)[0]]),
            'FlightNumber': flight_number,
            'AircraftID': random.choice([None, random.choice(aircraft_lookup)[0]]),
            'ActualDepartureDateTime': None,
            'ActualArrivalDateTime': None,
            'Status': random.choice(statuses),
            'DelayReasonCode': None,
            'CreatedAt': timestamp,
            'UpdatedAt': timestamp
        })

        leg_id_counter += 1

    return legs
